In [1]:
import os
import sys
import pandas as pd
import cx_Oracle
cx_Oracle.init_oracle_client(lib_dir=r"C:\Oracle\instantclient_21_6")

In [2]:
# FRAM001P
conn = cx_Oracle.connect(
    user="AD21236",
    password="L2022$Um",
    dsn="RACORAP08-SCAN.CORP.INTRANET/FRAM001P_PROD",
    encoding="UTF-8")

In [3]:
%%time
# each polygon will be associated with only one wire center, so only polygon id is needed in the filter

multigig_enabled_polygons_query = """

 WITH LQO AS (
            SELECT  DISTINCT LUID
                    ,WIRE_CENTER_ID
                    ,ADDRESS
                    ,ZIP_CODE
                    ,UNIT
                    ,UNI_POLYGONID        
            
            FROM    LQSMO.LU_QUAL_O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    AND (UNI_POLYGONID in (
                                          '1179499',  -- PHNXAZNE
                                          '1179547', '1179498', '1179121','1180067','1180494','1189253', -- PHNXAZSO
                                          '1178832', -- ARVDCOMA
                                          '1179849','1180344','1180107','1180306',-- MPLSMNBE 
                                          '1180028', '1178694', '1179569','1180333',-- MPLSMNBE 
                                          '1179862', '1179836', '1179604', '1179838', '1179887', '1180054', '1180222', -- MPLSMNBE
                                          '1178144', --PYLPWA01
                                          '1179687', '1179674','1180254', --GLDNCOMA
                                          '1180012','1180391', -- PTLDOR11
                                          '1179864', -- DNVRCOMA 
                                          '1179553','1179924','1179922', -- STPLMNBE 
                                          '1179688','1179157', '1179574',  -- MPLSMNTF
                                          '1180026','1180288','1179808','1179976', -- MPLSMNGE
                                          '1179583', -- DNVRCODC
                                          '1179774','1180265','1180031','1180237','1180468', -- CRYSMNCR
                                          '1180315','1189074','1189058', -- MRRYUTMA
                                          '1179708','1179714','1179719','1180072','1180075','1180089','1180095', -- TCSNAZMA
                                          '1180096','1179639','1179689','1179690','1179700','1179701','1179702', -- TCSNAZMA
                                          '1179703','1179704','1179718','1179793','1179803','1180092','550715', -- TCSNAZMA
                                          '1180094','1180507','1180076','1180079','1180492','1180497','1180486',  --TCSNAZMA
                                          '1180091','1179913','1189283',  --TCSNAZMA
                                          '1179641','1179914', --TCSNAZCR
                                          '1179840','1180355', --STTLWA04
                                          '1180086','1180111','1180327','1180436', --STTLWASU
                                          '1179576','1179672','1179971','1179977','1179979','1179981','1179987', --CLSPCOMA
                                          '1179988','1179989','1179994','1180005','1180006','1180041','1180123', --CLSPCOMA
                                          '1189130', --MLWKOR17
                                          '1179352','1180002','1189088', --MAGNUTNM
                                          '1179858','1180015','1180098','1180392','1180399', --SPKNWA01
                                          '1180027', --BRCTMNBC
                                          '1180491','1180509', --PHNXAZNO
                                          '1180488','1180496', --TCSNAZRN
                                          '1189200' --DNVRCOCL                          
                      
                                         )
                        OR WIRE_CENTER_ID = 'MPLSMNBE') -- per request, want all polygons in MPLSMNBE
        
        ),
        LQ AS (
        
        SELECT  Q.LUID
                ,Q.WIRE_CENTER_ID
                ,Q.ADDRESS
                ,Q.HNO
                ,Q.ST
                ,Q.UNIT
                ,Q.FLR
                ,Q.BLDG
                ,Q.COM
                ,Q.ZIP_CODE
                ,Q.TAR_CODE
                ,Q.BORN_ON_DATE
                ,Q.CAF_CENSUS_BLOCK
                ,Q.SERVING_TERMINAL
                ,Q.LU_XBOX
                ,Q.ASSOCIATED_TERMINAL_ID
                ,Q.REMOTE_TERMINAL_ID
                ,Q.REMOTE_TYPE
                ,Q.RES_BUS
                ,Q.LU_STAT

            FROM
                LU_QUAL Q
                JOIN LQO O
                    ON Q.WIRE_CENTER_ID = O.WIRE_CENTER_ID
        ),
        
        LC AS (
            
            SELECT * 
            FROM LU_CIRCUITS
        )
        
        SELECT DISTINCT
                O.LUID AS LU_QUAL_O_LUID
                ,O.UNI_POLYGONID
                ,A.*
                ,COALESCE(B.TELEPHONE_NUMBER,LCO.SUBSCRIBER) as TELEPHONE_OR_SUBSCRIBER
                ,B.LOOP_NAME
                ,B.CIRCUIT_IDENTIFIER
                ,B.SERVICE_CODE
                ,B.SERVICE_TYPE
                ,B.CS_USOC
                ,COALESCE(B.AL_USOC,LCO.HSI_SERVICE_PROFILE) as AL_USOC_OR_HSI_SERVICEPROFILE
                ,B.USER_ID
                ,B.TIME_STAMP
        FROM  LQO O
                JOIN LQ A
                    ON (TRIM(A.ADDRESS)||NVL(A.UNIT||A.FLR||A.BLDG,'.')=O.ADDRESS||NVL(REGEXP_REPLACE(O.UNIT, '^UNIT\s|^UNT\s|^APT\s|^FLR\s|BLDG\s', ''),'.'))
                    AND O.WIRE_CENTER_ID = A.WIRE_CENTER_ID
                LEFT JOIN LC B
                    ON  A.WIRE_CENTER_ID=B.WIRE_CENTER_ID
                    AND A.LUID=B.LUID
                LEFT JOIN LU_CIRCUITS_O LCO
                    ON O.LUID = LCO.LUID
                    AND O.WIRE_CENTER_ID = LCO.WIRE_CENTER_ID
                    AND LCO.SERVICE_FACS_STATUS = 'WKG'   --working customer base

"""

multigig_enabled_polygons_df = pd.read_sql(multigig_enabled_polygons_query, con=conn)

C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


CPU times: total: 609 ms
Wall time: 1min 10s


In [4]:
# check unique wire centers
multigig_enabled_polygons_df['WIRE_CENTER_ID'].value_counts()

TCSNAZMA    9521
MPLSMNBE    4912
CLSPCOMA    3128
MPLSMNGE    1769
PHNXAZSO    1712
CRYSMNCR    1396
MPLSMNTF     924
SPKNWA01     922
STPLMNBE     645
STTLWASU     484
GLDNCOMA     478
TCSNAZRN     462
DNVRCOMA     443
TCSNAZCR     403
DNVRCOCL     370
BRCTMNBC     366
MRRYUTMA     359
PYLPWA01     341
STTLWA04     322
MAGNUTNM     283
PHNXAZNO     249
PTLDOR11     234
PHNXAZNE     217
ARVDCOMA     206
DNVRCODC     175
MLWKOR17      93
Name: WIRE_CENTER_ID, dtype: int64

In [5]:
# separate data by wire center - manually add in wire centers

MPLSMNBE = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'MPLSMNBE']
PHNXAZSO = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'PHNXAZSO']
PYLPWA01 = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'PYLPWA01']
PHNXAZNE = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'PHNXAZNE']
ARVDCOMA = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'ARVDCOMA']
GLDNCOMA = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'GLDNCOMA']
DNVRCOMA = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'DNVRCOMA']
MPLSMNTF = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'MPLSMNTF']
STPLMNBE = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'STPLMNBE']
PTLDOR11 = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'PTLDOR11']
MPLSMNGE = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'MPLSMNGE']
DNVRCODC = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'DNVRCODC']
CRYSMNCR = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'CRYSMNCR']
TCSNAZMA = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'TCSNAZMA']
MRRYUTMA = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'MRRYUTMA']
TCSNAZCR = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'TCSNAZCR']
STTLWA04 = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'STTLWA04']
STTLWASU = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'STTLWASU']
CLSPCOMA = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'CLSPCOMA']
MLWKOR17 = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'MLWKOR17']
MAGNUTNM = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'MAGNUTNM']
SPKNWA01 = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'SPKNWA01']
BRCTMNBC = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'BRCTMNBC']
PHNXAZNO = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'PHNXAZNO']
TCSNAZRN = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'TCSNAZRN']
DNVRCOCL = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'DNVRCOCL']

In [6]:
import time

# export
TodaysDate = time.strftime("%m%d%y")
excelfilename = "Output/" + TodaysDate + "_LU_QUAL_slowsales.xlsx"

# add in new sheets manually if there are new wire centers turning up

with pd.ExcelWriter(excelfilename) as writer:  
    ARVDCOMA.to_excel(writer, sheet_name='ARVDCOMA', index = False)
    MPLSMNBE.to_excel(writer, sheet_name='MPLSMNBE_ALL_POLYGONS', index = False)
    PYLPWA01.to_excel(writer, sheet_name='PYLPWA01', index = False)
    PHNXAZSO.to_excel(writer, sheet_name='PHNXAZSO', index = False)
    PHNXAZNE.to_excel(writer, sheet_name='PHNXAZNE', index = False)
    GLDNCOMA.to_excel(writer, sheet_name='GLDNCOMA', index = False)
    DNVRCOMA.to_excel(writer, sheet_name='DNVRCOMA', index = False)
    MPLSMNTF.to_excel(writer, sheet_name='MPLSMNTF', index = False)
    STPLMNBE.to_excel(writer, sheet_name='STPLMNBE', index = False)
    PTLDOR11.to_excel(writer, sheet_name='PTLDOR11', index = False)
    MPLSMNGE.to_excel(writer, sheet_name='MPLSMNGE', index = False)
    DNVRCODC.to_excel(writer, sheet_name='DNVRCODC', index = False)
    CRYSMNCR.to_excel(writer, sheet_name='CRYSMNCR', index = False)
    TCSNAZMA.to_excel(writer, sheet_name='TCSNAZMA', index = False)
    TCSNAZCR.to_excel(writer, sheet_name='TCSNAZCR', index = False)
    MRRYUTMA.to_excel(writer, sheet_name='MRRYUTMA', index = False)
    STTLWA04.to_excel(writer, sheet_name='STTLWA04', index = False)
    STTLWASU.to_excel(writer, sheet_name='STTLWASU', index = False)
    CLSPCOMA.to_excel(writer, sheet_name='CLSPCOMA', index = False)
    MLWKOR17.to_excel(writer, sheet_name='MLWKOR17', index = False)
    MAGNUTNM.to_excel(writer, sheet_name='MAGNUTNM', index = False)
    SPKNWA01.to_excel(writer, sheet_name='SPKNWA01', index = False)
    BRCTMNBC.to_excel(writer, sheet_name='BRCTMNBC', index = False)
    PHNXAZNO.to_excel(writer, sheet_name='PHNXAZNO', index = False)
    TCSNAZRN.to_excel(writer, sheet_name='TCSNAZRN', index = False)
    DNVRCOCL.to_excel(writer, sheet_name='DNVRCOCL', index = False)